<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Viamagus_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -qU langchain langchain-community langchain_groq sentence-transformers  faiss-cpu torch langchain-huggingface

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

In [9]:
from huggingface_hub import notebook_login
notebook_login()

In [10]:
MAX_TOKENS = 225
EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
LLM_MODEL = ChatGroq(
    temperature=0,
    model="llama-3.1-70b-versatile",
    api_key="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"
)

In [11]:
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

RuntimeError: Failed to import transformers.models.clip.processing_clip because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline("text-generation", model=LLM_MODEL))

In [ ]:
history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]

In [ ]:
def prepare_documents(history):
    """
    Prepare the conversation history as documents for the vector store.

    Args:
    history (list): List of conversation messages.

    Returns:
    list: List of document strings.
    """
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    documents = text_splitter.create_documents(history)
    return documents

In [ ]:
def create_faiss_index(documents):
    """
    Create a FAISS index from the conversation history using LangChain.

    Args:
    documents (list): List of document strings.

    Returns:
    FAISS: FAISS vector store
    """
    vectorstore = FAISS.from_documents(documents, embedding_model)
    return vectorstore

In [ ]:
def setup_rag_chain(vectorstore):
    """
    Set up the Retrieval-Augmented Generation chain using LangChain.

    Args:
    vectorstore (FAISS): FAISS vector store

    Returns:
    RetrievalQA: RAG chain
    """
    prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

    {context}

    Question: {question}
    Answer:"""
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                           chain_type="stuff",
                                           retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
                                           chain_type_kwargs=chain_type_kwargs,
                                           return_source_documents=True)
    return qa_chain

In [ ]:
def test_final_prompt(qa_chain):
    """
    Test the final prompt by generating a response using the RAG chain.

    Args:
    qa_chain (RetrievalQA): RAG chain

    Returns:
    None
    """
    final_test_prompt = "Do you think it will help me stay fit?"

    try:
        result = qa_chain({"query": final_test_prompt})
        response = result['result']
        context = [doc.page_content for doc in result['source_documents']]

        print(f"Final Test Prompt: {final_test_prompt}")
        print(f"Context Referred: {context}")
        print(f"Final Test Prompt Response: {response}")

        # Validate token count
        total_tokens = len(tokenizer.encode(final_test_prompt)) + len(tokenizer.encode(response))
        print(f"Total tokens used: {total_tokens}")
        if total_tokens > MAX_TOKENS:
            print(f"Warning: Total tokens ({total_tokens}) exceed the maximum limit ({MAX_TOKENS}).")
    except Exception as e:
        print(f"Error generating response: {str(e)}")

In [ ]:
if __name__ == "__main__":
    # Prepare documents
    documents = prepare_documents(history)

    # Create FAISS index
    vectorstore = create_faiss_index(documents)

    # Set up RAG chain
    qa_chain = setup_rag_chain(vectorstore)

    # Test the final prompt
    test_final_prompt(qa_chain)